In [2]:
import h5py
import numpy as np
import os
import matplotlib.pyplot as plt
import time
from PIL import Image

In [10]:
data_dir = './Data'
total_np_x = list()
total_np_y = list()
total_np_log_x = list()
save_path = f'./Data/full_data.h5'
for dir in os.listdir(data_dir):
    count = 0
    for file in os.walk(os.path.join(data_dir, dir)):
        if len(file[2]) > 0:
            print(file[0].split('\\')[-1])

adenocarcinoma
large.cell.carcinoma
normal
squamous.cell.carcinoma
adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib
large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa
normal
squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa
adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib
large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa
normal
squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa


In [175]:
def convert_to_h5():
     data_dir = './Data'
     total_np_x = list()
     total_np_y = list()
     total_np_log_x = list()
     save_path = f'./Data/full_data.h5'
     for dir in os.listdir(data_dir):
          count = 0
          for file in os.walk(os.path.join(data_dir, dir)):
               # print(file[2])
               if len(file[2]) > 0:
                    print(file[0], len(file[2]))
                    for img_file in file[2]:
                         img_path = os.path.join(file[0], img_file)
                         img_f = Image.open(img_path)
                         img_f = img_f.convert('RGB')
                         orig_img = np.asarray(img_f)
                         resized = Image.fromarray(orig_img).resize(size=(60, 60))
                         resized_array = np.asarray(resized)

                         resized_log = Image.fromarray(resized_array).resize(size = (2, 2))
                         resized_log_squeezed = np.asarray(resized_log)
                         try:
                              resized_squeezed = resized_array.reshape(60*60*resized_array.shape[-1], 1)
                              # print(f"{resized_array.shape},  Normal - {resized_squeezed.shape}")
                              resized_log_squeezed = resized_log_squeezed.reshape(2*2*resized_array.shape[-1], 1)
                         except Exception as e:
                              print(e)
                         
                         # print(resized_squeezed.shape)
                         total_np_x.append(resized_squeezed)
                         total_np_log_x.append(resized_log_squeezed)
                         cancer_type = file[0].split('\\')[-1]
                         if 'adeno' in cancer_type or 'large' in cancer_type or 'squamous' in cancer_type:
                              total_np_y.append(1)
                         else:
                              total_np_y.append(0)
               
                         count += 1
               if count % 100:
                    print(count)
               # break
          # break

     total_np_x = np.asarray(np.squeeze(total_np_x, axis = 2))
     total_np_y = np.asarray(total_np_y)
     total_np_log_x = np.asarray(np.squeeze(total_np_log_x, axis = 2))
     # print(total_np_log_x.shape, total_np_x.shape, total_np_y.shape)
     with h5py.File(save_path, 'a')  as hf:
          grp_x = hf.create_dataset('data_x', data = total_np_x)
          grp_y = hf.create_dataset('data_y', data = total_np_y)
          grp_x_log = hf.create_dataset('data_x_log', data = total_np_log_x)

     
     # print('hdf5 file size: %d bytes'%os.path.getsize(save_path))

In [185]:
from sklearn.model_selection import train_test_split
def load_data():
    with h5py.File('./Data/full_data.h5', "r") as f:
        X = np.array(f["data_x"][:])
        Y = np.array(f["data_y"][:])
    train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, train_size=0.8, stratify=Y)    
    
    train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.25, train_size=0.75, stratify=train_y)  

    print(np.unique(valid_y, return_counts = True))
    # return train_x, train_y, valid_x, valid_y, test_x, test_y
    # assert train_x.shape == (6000, 10800, 1)
    # assert test_x.shape == (2000, 10800, 1)
    # assert valid_x.shape == (2000, 10800, 1)





In [186]:
load_data()

(array([0, 1]), array([ 43, 157], dtype=int64))


In [176]:
convert_to_h5()

./Data\test\adenocarcinoma 120
120
./Data\test\large.cell.carcinoma 51
171
./Data\test\normal 54
225
./Data\test\squamous.cell.carcinoma 90
315
./Data\train\adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib 195
195
./Data\train\large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa 115
310
./Data\train\normal 148
458
./Data\train\squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa 155
613
./Data\valid\adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib 23
23
./Data\valid\large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa 21
44
./Data\valid\normal 13
57
./Data\valid\squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa 15
72
